<a href="https://colab.research.google.com/github/asdfasdf001234/2024-1-MLPRJ/blob/main/Resnet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import os
import time
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
import seaborn as sns

In [34]:
# pretrained 관련
import torch
import torchvision.transforms as v2
from torchvision import models

In [35]:
import torch
import numpy as np
import random

def set_random_seed(seed_value):
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    np.random.seed(seed_value)
    random.seed(seed_value)

# Set a random seed value
seed_value = 42
set_random_seed(seed_value)


# data 준비
- personal color data (train/val/test)
- data: 1-봄 웜 / 2-여름 쿨 / 3-가을 웜 / 4-겨울 쿨
- labeling: 0-봄 웜 / 1-여름 쿨 / 2-가을 웜 / 3-겨울 쿨

In [36]:
from torch.utils.data import Dataset , DataLoader
from torchvision import datasets
from torchvision.transforms import v2

In [37]:
from PIL import Image
from glob import glob

In [38]:
data_path = "./drive/MyDrive/Colab Notebooks/MLPRJ/Data"

In [39]:
import pandas as pd
import os
from glob import glob

def create_dataframe(data_path, label_list, data_type):
    df = pd.DataFrame({"path": [], "label": [], "class_id": []})
    img_list = glob(os.path.join(data_path, '*.jpg'))

    for img in img_list:
      file_name = os.path.splitext(os.path.basename(img))[0]
      label_index = int(file_name[0]) - 1
      if label_index == 0 or label_index == 2:   #웜톤
        label_index = 0                          #0으로 통일
      else:                                      #쿨톤
        label_index = 1                          #1으로 통일
      if 0 <= label_index < len(label_list):
        label = label_list[label_index]
        new_data = pd.DataFrame({"path": [img], "label": [label], "class_id": [label_index]})
        df = pd.concat([df, new_data], ignore_index=True)


    df[["path"]] = df[["path"]].astype(str)
    df[["label"]] = df[["label"]].astype(str)
    df[["class_id"]] = df[["class_id"]].astype(int)

    return df

'''
#웜쿨 분류 코드 (2분류 체계)
    for img in img_list:
      file_name = os.path.splitext(os.path.basename(img))[0]
      label_index = int(file_name[0]) - 1
      if label_index == 0 or label_index == 2:   #웜톤
        label_index = 0                          #0으로 통일
      else:                                      #쿨톤
        label_index = 1                          #1으로 통일
      if 0 <= label_index < len(label_list):
        label = label_list[label_index]
        new_data = pd.DataFrame({"path": [img], "label": [label], "class_id": [label_index]})
        df = pd.concat([df, new_data], ignore_index=True)
#수정 코드 끝
'''


'''
기존 코드 (4분류 체계)
    for img in img_list:
      file_name = os.path.splitext(os.path.basename(img))[0]
      label_index = int(file_name[0]) - 1  #진영님 코드와 동일
      if 0 <= label_index < len(label_list):
        label = label_list[label_index]
        new_data = pd.DataFrame({"path": [img], "label": [label], "class_id": [label_index]})
        df = pd.concat([df, new_data], ignore_index=True)
'''

'\n기존 코드 (4분류 체계)\n    for img in img_list:\n      file_name = os.path.splitext(os.path.basename(img))[0]\n      label_index = int(file_name[0]) - 1  #진영님 코드와 동일\n      if 0 <= label_index < len(label_list):\n        label = label_list[label_index]\n        new_data = pd.DataFrame({"path": [img], "label": [label], "class_id": [label_index]})\n        df = pd.concat([df, new_data], ignore_index=True)\n'

In [40]:
# 기존 example 코드
train_path = data_path + '/train'
valid_path = data_path + '/val'
test_path = data_path + '/test'
label_list = ['spring', 'summer', 'autumn', 'winter']
#label_list = ['warm', 'cool']

train_df = create_dataframe(train_path, label_list, 'training')
val_df = create_dataframe(valid_path, label_list, 'validation')
test_df = create_dataframe(test_path, label_list, 'testing')

In [41]:
#2분류 코드
train_path = data_path + '/train'
valid_path = data_path + '/val'
test_path = data_path + '/test'
label_list = ['warm', 'cool']

train_df = create_dataframe(train_path, label_list, 'training')
val_df = create_dataframe(valid_path, label_list, 'validation')
test_df = create_dataframe(test_path, label_list, 'testing')

In [42]:
print(f"train_data: {len(train_df)}")
print(f"val_data:{len(val_df)}")
print(f"test_data:{len(test_df)}")

train_data: 413
val_data:137
test_data:137


In [43]:
#데이터 출력
print("Train DataFrame:")
print(train_df.head())
print("Validation DataFrame:")
print(val_df.head())
print("Test DataFrame:")
print(test_df.head())

Train DataFrame:
                                                path label  class_id
0  ./drive/MyDrive/Colab Notebooks/MLPRJ/Data/tra...  warm         0
1  ./drive/MyDrive/Colab Notebooks/MLPRJ/Data/tra...  warm         0
2  ./drive/MyDrive/Colab Notebooks/MLPRJ/Data/tra...  warm         0
3  ./drive/MyDrive/Colab Notebooks/MLPRJ/Data/tra...  warm         0
4  ./drive/MyDrive/Colab Notebooks/MLPRJ/Data/tra...  warm         0
Validation DataFrame:
                                                path label  class_id
0  ./drive/MyDrive/Colab Notebooks/MLPRJ/Data/val...  warm         0
1  ./drive/MyDrive/Colab Notebooks/MLPRJ/Data/val...  warm         0
2  ./drive/MyDrive/Colab Notebooks/MLPRJ/Data/val...  warm         0
3  ./drive/MyDrive/Colab Notebooks/MLPRJ/Data/val...  warm         0
4  ./drive/MyDrive/Colab Notebooks/MLPRJ/Data/val...  warm         0
Test DataFrame:
                                                path label  class_id
0  ./drive/MyDrive/Colab Notebooks/MLPRJ/Data/te

In [44]:
class BaseDataset(torch.utils.data.Dataset):
    def __init__(self , dataframe , transforms_):
        self.df = dataframe
        self.transforms_ = transforms_

    def __len__(self):
        return len(self.df)

    def __getitem__(self ,index):
        img_path = self.df.iloc[index]['path']
        img = Image.open(img_path).convert("RGB")
        transformed_img = self.transforms_(img)
        class_id = self.df.iloc[index]['class_id']
        return transformed_img , class_id

In [58]:
Transforms = v2.Compose([
    v2.Resize((224,224)),
    v2.RandomRotation(degrees=10),
    v2.RandomHorizontalFlip(p=0.8),
    v2.ScaleJitter(target_size=(224,224)),
    v2.RandomAffine(degrees=45),
    #v2.RandomErasing(),
    v2.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5.)),
    v2.ElasticTransform(alpha=250.0),
    #v2.ColorJitter(0.5, 0.5),
    #v2.RandomResizedCrop(size=(224, 224), antialias=True),

    v2.Resize((224,224)),
    v2.PILToTensor(),
    v2.ToDtype(torch.float32),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

In [59]:
BATCH_SIZE = 30
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [60]:
#get dataloader

train_dataset = BaseDataset(train_df, Transforms) # train_transforms
val_dataset = BaseDataset(val_df, Transforms)
test_dataset = BaseDataset(test_df, Transforms)

In [61]:
train_loader = DataLoader(train_dataset , batch_size=BATCH_SIZE , shuffle = True)
val_loader = DataLoader(val_dataset , batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset , batch_size=BATCH_SIZE)

In [62]:
resnet50 = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

In [63]:
print(resnet50)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [64]:
import torch.nn as nn
class MyRes50(nn.Module):
    def __init__(self, pretrained_model):
        super(MyRes50, self).__init__()
        self.backbone = pretrained_model

        #for param in self.backbone.parameters():
        #  param.requires_grad = True

        for name, param in self.backbone.named_parameters():
            if 'fc' in name:
                param.requires_grad = True
            elif 'layer4' in name:
                param.requires_grad = True
            else:
                param.requires_grad = False

        self.dropout = nn.Dropout(0.25)


        self.fc1 = nn.Linear(1000, 256)
        self.fc2 = nn.Linear(256, 32)
        self.fc3 = nn.Linear(32, 2)

    def forward(self, x):
        x = self.backbone(x)
        x = self.dropout(x)
        x = self.dropout(self.fc1(x))
        x = self.dropout(self.fc2(x))
        x = self.fc3(x)
        return x


In [65]:
myres50 = MyRes50(resnet50)

In [66]:
myres50.cuda()

MyRes50(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
         

In [67]:
seed_value = 42
set_random_seed(seed_value)

EPOCHS = 100
logs = {"train_loss":[] , "train_acc":[] , "val_loss":[] , "val_acc":[]}

if os.path.exists('checkpoints') == False:
    os.mkdir('checkpoints')

criterion = nn.CrossEntropyLoss()


optimizer = optim.AdamW(resnet50.parameters(), lr=0.01)
Cosine_lr_scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)


patience = 10
counter = 0
best_loss = np.inf

In [68]:
def train(dataloader , model , loss_fn , optimizer , lr_scheduler):
    size = 0
    num_batches = len(dataloader)

    model.train()
    epoch_loss , epoch_correct = 0 , 0

    for i ,(data_ , target_) in enumerate(dataloader):

        data = data_.to(device)
        target = target_.to(device)

        pred = model(data)
        _, pred_max = torch.max(pred, 1)

        loss = loss_fn(pred, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_correct += (pred_max == target).type(torch.float).sum().item()
        size += target.size(0)


    train_acc = epoch_correct/size
    lr_scheduler.step()

    return train_acc , epoch_loss / num_batches


def test(dataloader , model , loss_fn):
    size = 0
    num_baches = len(dataloader)
    epoch_loss , epoch_correct= 0 ,0
    with torch.no_grad(): # grad 연산 X
        model.eval() # evaluation dropout 연산시
        for i, (data_ , target_) in enumerate(dataloader):

            data = data_.to(device)
            target = target_.to(device)

            pred = model(data)
            _, pred_max = torch.max(pred, 1)

            loss = loss_fn(pred, target)

            epoch_loss += loss.item()
            epoch_correct += (pred_max == target).type(torch.float).sum().item()
            size += target.size(0)

    test_acc = epoch_correct/size

    return test_acc  , epoch_loss / num_baches

In [69]:
for epoch in tqdm(range(EPOCHS)):
    train_acc , train_loss = train(train_loader ,
                                   myres50 ,
                                   criterion ,
                                   optimizer ,
                                   Cosine_lr_scheduler)

    val_acc , val_loss = test(val_loader , myres50 , criterion)
    print(f'epoch:{epoch} \
    train_loss = {train_loss:.4f} , train_acc:{train_acc:.4f} \
    val_loss = {val_loss:.4f} , val_acc:{val_acc:.4f} \
    learning rate: {optimizer.param_groups[0]["lr"]}')
    logs['train_loss'].append(train_loss)
    logs['train_acc'].append(train_acc)
    logs['val_loss'].append(val_loss)
    logs['val_acc'].append(val_acc)


    if val_loss < best_loss:
        counter = 0
        best_loss = val_loss
        #torch.save(myres50.state_dict() , "checkpoints/myvgg_16_best.pth")
    else:
        counter+=1

#test_acc, val_loss = test(test_loader, myres50, criterion)
#print(test_acc)

    if counter >= patience:
        test_acc , val_loss = test(test_loader , myres50 , criterion)
        print("Early stop !")
        print(test_acc)
        break



  1%|          | 1/100 [01:40<2:46:13, 100.75s/it]

epoch:0     train_loss = 1.6786 , train_acc:0.5932     val_loss = 120.6477 , val_acc:0.3650     learning rate: 0.009000000000000001


  2%|▏         | 2/100 [03:31<2:53:58, 106.52s/it]

epoch:1     train_loss = 0.7768 , train_acc:0.5932     val_loss = 2.7054 , val_acc:0.6350     learning rate: 0.008100000000000001


  3%|▎         | 3/100 [05:18<2:52:58, 106.99s/it]

...

epoch:17     train_loss = 0.5085 , train_acc:0.7337     val_loss = 0.6286 , val_acc:0.6423     learning rate: 0.0015009463529699924


 19%|█▉        | 19/100 [34:10<2:27:20, 109.14s/it]

epoch:18     train_loss = 0.5007 , train_acc:0.7869     val_loss = 0.5999 , val_acc:0.6642     learning rate: 0.0013508517176729932
epoch:19     train_loss = 0.4901 , train_acc:0.7506     val_loss = 0.8294 , val_acc:0.6788     learning rate: 0.001215766545905694


 19%|█▉        | 19/100 [37:04<2:38:03, 117.08s/it]

Early stop !
0.708029197080292


In [70]:
test_acc , val_loss = test(test_loader , myres50 , criterion)
print(test_acc)

0.7737226277372263
